In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.measure import label, regionprops


#### pre-processing and feature extraction functions

In [5]:
# thresholding
def thresh(a):
    t = np.zeros(np.shape(a), dtype='uint8')
    t[a < 100] = 0  # black
    t[a > 100] = 1  # white
    return t
   
# connected regions labeling
def label_images(a):
    image_label = label(a, background=-1)
    props = regionprops(image_label)
    return props

# blackness ration feature
def blackness_ratio(props):
    black_ration = props[1].area / props[1].bbox_area
    return black_ration

# aspect ratio
def aspect_ratio(props):
    asp_ratio = (props[1].bbox[3] - props[1].bbox[1] + 1) / (props[1].bbox[2] - props[1].bbox[0] + 1)
    return asp_ratio

# number of holes
def num_holes(props):
    holes = len(props) - 2
    return holes

In [6]:
def all_together(a):
    thresh_array = np.zeros(np.shape(a), dtype='uint8')
    thresh_array = thresh(a)
    
    # pre-allocation 
    
    # image_label = np.zeros(np.shape(a), dtype='uint8')
    black_ratio_result = np.zeros((60000,), dtype='uint32')
    aspect_ratio_result = np.zeros((60000,), dtype='uint32')
    num_holes_result = np.zeros((60000,), dtype='int')
    
    for i in range(0, 60000):
        print('start {}...'.format(i))
        props = label_images(thresh_array[i,:,:])
        black_ratio_result[i] = blackness_ratio(props)
        aspect_ratio_result[i] = aspect_ratio(props)
        num_holes_result[i] = num_holes(props)
        print('{} finished'.format(i))
        %clear 
    return black_ratio_result, aspect_ratio_result, num_holes_result
        
        

In [7]:
# pre-alloc
black_ratio_r = np.zeros((60000,), dtype='uint32')
aspect_ratio_r = np.zeros((60000,), dtype='uint32')
num_holes_r = np.zeros((60000,), dtype='int')


# data-loading
images = np.load('/mnt/e/Pattern Recognition/DigitRecognizer/data/images.npy')
labels = np.load('/mnt/e/Pattern Recognition/DigitRecognizer/data/labels.npy')

black_ratio_r[:,], aspect_ratio_r[:,], num_holes_r[:,] = all_together(images[:,:,:])

start 0...
0 finished
start 1...
1 finished
start 2...
2 finished
start 3...
3 finished
start 4...
4 finished
start 5...
5 finished
start 6...
6 finished
start 7...
7 finished
start 8...
8 finished
start 9...
9 finished
start 10...
10 finished
start 11...
11 finished
start 12...
12 finished
start 13...
13 finished
start 14...
14 finished
start 15...
15 finished
start 16...
16 finished
start 17...
17 finished
start 18...
18 finished
start 19...
19 finished
start 20...
20 finished
start 21...
21 finished
start 22...
22 finished
start 23...
23 finished
start 24...
24 finished
start 25...
25 finished
start 26...
26 finished
start 27...
27 finished
start 28...
28 finished
start 29...
29 finished
start 30...
30 finished
start 31...
31 finished
start 32...
32 finished
start 33...
33 finished
start 34...
34 finished
start 35...
35 finished
start 36...
36 finished
start 37...
37 finished
start 38...
38 finished
start 39...
39 finished
start 40...
40 finished
start 41...
41 finished
start 42...


691 finished
start 692...
692 finished
start 693...
693 finished
start 694...
694 finished
start 695...
695 finished
start 696...
696 finished
start 697...
697 finished
start 698...
698 finished
start 699...
699 finished
start 700...
700 finished
start 701...
701 finished
start 702...
702 finished
start 703...
703 finished
start 704...
704 finished
start 705...
705 finished
start 706...
706 finished
start 707...
707 finished
start 708...
708 finished
start 709...
709 finished
start 710...
710 finished
start 711...
711 finished
start 712...
712 finished
start 713...
713 finished
start 714...
714 finished
start 715...
715 finished
start 716...
716 finished
start 717...
717 finished
start 718...
718 finished
start 719...
719 finished
start 720...
720 finished
start 721...
721 finished
start 722...
722 finished
start 723...
723 finished
start 724...
724 finished
start 725...
725 finished
start 726...
726 finished
start 727...
727 finished
start 728...
728 finished
start 729...
729 finished

1264 finished
start 1265...
1265 finished
start 1266...
1266 finished
start 1267...
1267 finished
start 1268...
1268 finished
start 1269...
1269 finished
start 1270...
1270 finished
start 1271...
1271 finished
start 1272...
1272 finished
start 1273...
1273 finished
start 1274...
1274 finished
start 1275...
1275 finished
start 1276...
1276 finished
start 1277...
1277 finished
start 1278...
1278 finished
start 1279...
1279 finished
start 1280...
1280 finished
start 1281...
1281 finished
start 1282...
1282 finished
start 1283...
1283 finished
start 1284...
1284 finished
start 1285...
1285 finished
start 1286...
1286 finished
start 1287...
1287 finished
start 1288...
1288 finished
start 1289...
1289 finished
start 1290...
1290 finished
start 1291...
1291 finished
start 1292...
1292 finished
start 1293...
1293 finished
start 1294...
1294 finished
start 1295...
1295 finished
start 1296...
1296 finished
start 1297...
1297 finished
start 1298...
1298 finished
start 1299...
1299 finished
start 

1861 finished
start 1862...
1862 finished
start 1863...
1863 finished
start 1864...
1864 finished
start 1865...
1865 finished
start 1866...
1866 finished
start 1867...
1867 finished
start 1868...
1868 finished
start 1869...
1869 finished
start 1870...
1870 finished
start 1871...
1871 finished
start 1872...
1872 finished
start 1873...
1873 finished
start 1874...
1874 finished
start 1875...
1875 finished
start 1876...
1876 finished
start 1877...
1877 finished
start 1878...
1878 finished
start 1879...
1879 finished
start 1880...
1880 finished
start 1881...
1881 finished
start 1882...
1882 finished
start 1883...
1883 finished
start 1884...
1884 finished
start 1885...
1885 finished
start 1886...
1886 finished
start 1887...
1887 finished
start 1888...
1888 finished
start 1889...
1889 finished
start 1890...
1890 finished
start 1891...
1891 finished
start 1892...
1892 finished
start 1893...
1893 finished
start 1894...
1894 finished
start 1895...
1895 finished
start 1896...
1896 finished
start 

start 2441...
2441 finished
start 2442...
2442 finished
start 2443...
2443 finished
start 2444...
2444 finished
start 2445...
2445 finished
start 2446...
2446 finished
start 2447...
2447 finished
start 2448...
2448 finished
start 2449...
2449 finished
start 2450...
2450 finished
start 2451...
2451 finished
start 2452...
2452 finished
start 2453...
2453 finished
start 2454...
2454 finished
start 2455...
2455 finished
start 2456...
2456 finished
start 2457...
2457 finished
start 2458...
2458 finished
start 2459...
2459 finished
start 2460...
2460 finished
start 2461...
2461 finished
start 2462...
2462 finished
start 2463...
2463 finished
start 2464...
2464 finished
start 2465...
2465 finished
start 2466...
2466 finished
start 2467...
2467 finished
start 2468...
2468 finished
start 2469...
2469 finished
start 2470...
2470 finished
start 2471...
2471 finished
start 2472...
2472 finished
start 2473...
2473 finished
start 2474...
2474 finished
start 2475...
2475 finished
start 2476...
2476 f

...

57763 finished
start 57764...
57764 finished
start 57765...
57765 finished
start 57766...
57766 finished
start 57767...
57767 finished
start 57768...
57768 finished
start 57769...
57769 finished
start 57770...
57770 finished
start 57771...
57771 finished
start 57772...
57772 finished
start 57773...
57773 finished
start 57774...
57774 finished
start 57775...
57775 finished
start 57776...
57776 finished
start 57777...
57777 finished
start 57778...
57778 finished
start 57779...
57779 finished
start 57780...
57780 finished
start 57781...
57781 finished
start 57782...
57782 finished
start 57783...
57783 finished
start 57784...
57784 finished
start 57785...
57785 finished
start 57786...
57786 finished
start 57787...
57787 finished
start 57788...
57788 finished
start 57789...
57789 finished
start 57790...
57790 finished
start 57791...
57791 finished
start 57792...
57792 finished
start 57793...
57793 finished
start 57794...
57794 finished
start 57795...
57795 finished
start 57796...
57796 fini

58436 finished
start 58437...
58437 finished
start 58438...
58438 finished
start 58439...
58439 finished
start 58440...
58440 finished
start 58441...
58441 finished
start 58442...
58442 finished
start 58443...
58443 finished
start 58444...
58444 finished
start 58445...
58445 finished
start 58446...
58446 finished
start 58447...
58447 finished
start 58448...
58448 finished
start 58449...
58449 finished
start 58450...
58450 finished
start 58451...
58451 finished
start 58452...
58452 finished
start 58453...
58453 finished
start 58454...
58454 finished
start 58455...
58455 finished
start 58456...
58456 finished
start 58457...
58457 finished
start 58458...
58458 finished
start 58459...
58459 finished
start 58460...
58460 finished
start 58461...
58461 finished
start 58462...
58462 finished
start 58463...
58463 finished
start 58464...
58464 finished
start 58465...
58465 finished
start 58466...
58466 finished
start 58467...
58467 finished
start 58468...
58468 finished
start 58469...
58469 fini

59133 finished
start 59134...
59134 finished
start 59135...
59135 finished
start 59136...
59136 finished
start 59137...
59137 finished
start 59138...
59138 finished
start 59139...
59139 finished
start 59140...
59140 finished
start 59141...
59141 finished
start 59142...
59142 finished
start 59143...
59143 finished
start 59144...
59144 finished
start 59145...
59145 finished
start 59146...
59146 finished
start 59147...
59147 finished
start 59148...
59148 finished
start 59149...
59149 finished
start 59150...
59150 finished
start 59151...
59151 finished
start 59152...
59152 finished
start 59153...
59153 finished
start 59154...
59154 finished
start 59155...
59155 finished
start 59156...
59156 finished
start 59157...
59157 finished
start 59158...
59158 finished
start 59159...
59159 finished
start 59160...
59160 finished
start 59161...
59161 finished
start 59162...
59162 finished
start 59163...
59163 finished
start 59164...
59164 finished
start 59165...
59165 finished
start 59166...
59166 fini

59814 finished
start 59815...
59815 finished
start 59816...
59816 finished
start 59817...
59817 finished
start 59818...
59818 finished
start 59819...
59819 finished
start 59820...
59820 finished
start 59821...
59821 finished
start 59822...
59822 finished
start 59823...
59823 finished
start 59824...
59824 finished
start 59825...
59825 finished
start 59826...
59826 finished
start 59827...
59827 finished
start 59828...
59828 finished
start 59829...
59829 finished
start 59830...
59830 finished
start 59831...
59831 finished
start 59832...
59832 finished
start 59833...
59833 finished
start 59834...
59834 finished
start 59835...
59835 finished
start 59836...
59836 finished
start 59837...
59837 finished
start 59838...
59838 finished
start 59839...
59839 finished
start 59840...
59840 finished
start 59841...
59841 finished
start 59842...
59842 finished
start 59843...
59843 finished
start 59844...
59844 finished
start 59845...
59845 finished
start 59846...
59846 finished
start 59847...
59847 fini

IndexError: too many indices for array